In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# ✅ Cell 1: Import thư viện cần thiết

In [ ]:
!pip install -q segmentation-models-pytorch albumentations timm

import os
import cv2
import random
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import segmentation_models_pytorch as smp

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)


# Cell 2: Định nghĩa cấu hình chung + đọc danh sách (image, mask) cho train/val (no augmentation)

In [ ]:
# thay thế
# Cell 2: Cấu hình + tự động xác định DATA_DIR và thư mục masks linh hoạt (no augmentation)

class CFG:
    IMG_SIZE = 512
    EPOCHS = 80
    BATCH_SIZE = 4
    LR = 1e-4
    NUM_WORKERS = 2
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# 1) Tìm thư mục gốc có cấu trúc .../train/images
ROOT_SEARCH = "/kaggle/input/thesis"

candidate_roots = []
for root, dirs, files in os.walk(ROOT_SEARCH):
    norm_root = root.replace("\\", "/")
    if norm_root.endswith("/train/images"):
        dataset_root = norm_root.rsplit("/train/images", 1)[0]
        candidate_roots.append(dataset_root)

candidate_roots = sorted(set(candidate_roots))
print("Candidate DATA_DIR found:", candidate_roots)

if len(candidate_roots) == 0:
    raise FileNotFoundError(
        "Không tìm thấy thư mục train/images trong /kaggle/input/thesis. Kiểm tra lại cấu trúc dataset."
    )

DATA_DIR = candidate_roots[0]
print("Using DATA_DIR:", DATA_DIR)

# 2) Hàm tìm thư mục mask phù hợp (mask / masks / labels / segmentation...)
def find_mask_dir(split: str):
    split_dir = os.path.join(DATA_DIR, split)
    if not os.path.exists(split_dir):
        raise FileNotFoundError(f"Không tồn tại thư mục split: {split_dir}")
    
    # Ưu tiên thư mục tên chứa 'mask' hoặc 'label'
    candidates = []
    for name in os.listdir(split_dir):
        path = os.path.join(split_dir, name)
        if os.path.isdir(path):
            low = name.lower()
            if "mask" in low or "label" in low or "seg" in low:
                candidates.append(path)
    
    # Nếu không tìm thấy, fallback về 'masks'
    if not candidates:
        default_dir = os.path.join(split_dir, "masks")
        if os.path.isdir(default_dir):
            candidates.append(default_dir)
    
    if not candidates:
        print(f"[WARN] Không tìm thấy thư mục mask/label trong: {split_dir}")
        return None
    
    # Chọn candidate đầu tiên
    mask_dir = sorted(candidates)[0]
    print(f"{split} mask_dir:", mask_dir)
    return mask_dir

def get_image_mask_pairs(split: str):
    image_dir = os.path.join(DATA_DIR, split, "images")
    if not os.path.isdir(image_dir):
        print(f"[WARN] Không tìm thấy {image_dir}")
        return []
    
    mask_dir = find_mask_dir(split)
    if mask_dir is None:
        return []
    
    image_paths = sorted(glob(os.path.join(image_dir, "*")))
    exts = [".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp"]
    pairs = []

    for img_path in image_paths:
        base = os.path.splitext(os.path.basename(img_path))[0]
        found_mask = None
        for ext in exts:
            cand = os.path.join(mask_dir, base + ext)
            if os.path.exists(cand):
                found_mask = cand
                break
        if found_mask is not None:
            pairs.append((img_path, found_mask))

    print(f"{split} - found {len(pairs)} pairs")
    # In thử 3 cặp đầu để kiểm tra
    for p in pairs[:3]:
        print(" sample:", p[0], "->", p[1])
    return pairs

train_pairs = get_image_mask_pairs("train")
validation_pairs = get_image_mask_pairs("validation")

print("Total train:", len(train_pairs), "| Total val:", len(validation_pairs))


In [ ]:
# Cell fix môi trường: khôi phục NumPy + Matplotlib tương thích (xử lý lỗi _ARRAY_API / multiarray)

!pip install -q --force-reinstall "numpy>=2.0.0,<2.3.0" "matplotlib>=3.8.0,<3.10.0"

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print("NumPy version:", np.__version__)
print("Matplotlib version:", matplotlib.__version__)

In [ ]:
import matplotlib.pyplot as plt

split_names = ["Train", "Validation"]
split_counts = [len(train_pairs), len(validation_pairs)]

plt.figure(figsize=(5, 4))
plt.bar(split_names, split_counts)
plt.title("Dataset Split Overview (No Augmentation)")
plt.ylabel("Number of Image-Mask Pairs")
plt.grid(axis="y", linestyle="--", alpha=0.6)
for i, v in enumerate(split_counts):
    plt.text(i, v + 5, str(v), ha='center', fontsize=10)
plt.show()

# Cell 3: Định nghĩa Dataset + DataLoader với transforms cơ bản (no augmentation)

In [ ]:
class DFUSegDataset(Dataset):
    def __init__(self, pairs, img_size=CFG.IMG_SIZE):
        self.pairs = pairs
        self.img_size = img_size
        self.transform = A.Compose([
            A.Resize(self.img_size, self.img_size),
            A.Normalize(mean=(0.485, 0.456, 0.406),
                        std=(0.229, 0.224, 0.225)),
            ToTensorV2(),
        ])

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img_path, mask_path = self.pairs[idx]

        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # chuẩn về nhị phân 0/1
        mask = (mask > 0).astype("float32")

        augmented = self.transform(image=image, mask=mask)
        image = augmented["image"]
        mask = augmented["mask"].unsqueeze(0)  # (1, H, W)

        return image, mask

# dùng train_pairs và val_pairs đã tạo ở Cell 2
train_dataset = DFUSegDataset(train_pairs, img_size=CFG.IMG_SIZE)
val_dataset = DFUSegDataset(validation_pairs, img_size=CFG.IMG_SIZE)

train_loader = DataLoader(
    train_dataset,
    batch_size=CFG.BATCH_SIZE,
    shuffle=True,
    num_workers=CFG.NUM_WORKERS,
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CFG.BATCH_SIZE,
    shuffle=False,
    num_workers=CFG.NUM_WORKERS,
    pin_memory=True,
)

print("Train batches:", len(train_loader), "| Val batches:", len(val_loader))


# Cell 4: Định nghĩa hàm loss BCE + Dice và các metric cơ bản (Dice, IoU, Precision, Recall)

In [ ]:
def dice_coef(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    inter = (pred * target).sum(dim=(2, 3))
    union = pred.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
    dice = (2.0 * inter + smooth) / (union + smooth)
    return dice.mean().item()

def iou_coef(pred, target, smooth=1e-5):
    pred = torch.sigmoid(pred)
    pred = (pred > 0.5).float()
    inter = (pred * target).sum(dim=(2, 3))
    union = (pred + target - pred * target).sum(dim=(2, 3))
    iou = (inter + smooth) / (union + smooth)
    return iou.mean().item()

def bce_dice_loss(pred, target, smooth=1e-5):
    bce = F.binary_cross_entropy_with_logits(pred, target)
    pred_sig = torch.sigmoid(pred)
    inter = (pred_sig * target).sum(dim=(2, 3))
    union = pred_sig.sum(dim=(2, 3)) + target.sum(dim=(2, 3))
    dice = 1 - ((2 * inter + smooth) / (union + smooth))
    return bce + dice.mean()

print("✅ Defined: dice_coef, iou_coef, and bce_dice_loss")


# Cell 5: Định nghĩa hàm train và validation cho 1 epoch (no augmentation)

In [ ]:
import gc

def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss, total_dice, total_iou = 0.0, 0.0, 0.0

    for images, masks in tqdm(loader, desc="Training", leave=False):
        images, masks = images.to(CFG.DEVICE), masks.to(CFG.DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = bce_dice_loss(outputs, masks)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_dice += dice_coef(outputs, masks)
        total_iou += iou_coef(outputs, masks)

    mean_loss = total_loss / len(loader)
    mean_dice = total_dice / len(loader)
    mean_iou = total_iou / len(loader)

    return mean_loss, mean_dice, mean_iou


@torch.no_grad()
def valid_one_epoch(model, loader):
    model.eval()
    total_loss, total_dice, total_iou = 0.0, 0.0, 0.0

    for images, masks in tqdm(loader, desc="Validation", leave=False):
        images, masks = images.to(CFG.DEVICE), masks.to(CFG.DEVICE)

        outputs = model(images)
        loss = bce_dice_loss(outputs, masks)

        total_loss += loss.item()
        total_dice += dice_coef(outputs, masks)
        total_iou += iou_coef(outputs, masks)

    mean_loss = total_loss / len(loader)
    mean_dice = total_dice / len(loader)
    mean_iou = total_iou / len(loader)

    return mean_loss, mean_dice, mean_iou


def train_model(model, model_name, num_epochs=CFG.EPOCHS):
    model = model.to(CFG.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=5, verbose=True
    )

    best_dice = 0.0
    best_loss = np.inf

    history = {
        "train_loss": [],
        "val_loss": [],
        "train_dice": [],
        "val_dice": [],
        "train_iou": [],
        "val_iou": [],
    }

    save_path = f"{model_name}_best_no_aug.pth"

    for epoch in range(1, num_epochs + 1):
        print(f"\n===== {model_name} | Epoch {epoch}/{num_epochs} =====")

        torch.cuda.empty_cache()
        gc.collect()

        train_loss, train_dice, train_iou = train_one_epoch(model, train_loader, optimizer)
        val_loss, val_dice, val_iou = valid_one_epoch(model, val_loader)

        scheduler.step(val_loss)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_dice"].append(train_dice)
        history["val_dice"].append(val_dice)
        history["train_iou"].append(train_iou)
        history["val_iou"].append(val_iou)

        print(
            f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
            f"Train Dice: {train_dice:.4f} | Val Dice: {val_dice:.4f} | "
            f"Train IoU: {train_iou:.4f} | Val IoU: {val_iou:.4f}"
        )

        if val_dice > best_dice:
            best_dice = val_dice
            best_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f"✅ Saved best model: {save_path} (Val Dice: {best_dice:.4f})")

    print(f"\nBest Val Dice for {model_name}: {best_dice:.4f} (Loss: {best_loss:.4f})")
    return model, history

# Cell 6: Định nghĩa hàm train_model()

In [ ]:
def train_model(model, model_name, num_epochs=CFG.EPOCHS):
    model = model.to(CFG.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=CFG.LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=5, verbose=True
    )

    best_dice = 0.0
    best_loss = np.inf

    history = {
        "train_loss": [],
        "val_loss": [],
        "train_dice": [],
        "val_dice": [],
        "train_iou": [],
        "val_iou": [],
    }

    save_path = f"{model_name}_best_no_aug.pth"

    for epoch in range(1, num_epochs + 1):
        print(f"\n===== {model_name} | Epoch {epoch}/{num_epochs} =====")

        torch.cuda.empty_cache()
        gc.collect()

        # nhận đủ 3 giá trị
        train_loss, train_dice, train_iou = train_one_epoch(model, train_loader, optimizer)
        val_loss, val_dice, val_iou = valid_one_epoch(model, val_loader)

        scheduler.step(val_loss)

        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["train_dice"].append(train_dice)
        history["val_dice"].append(val_dice)
        history["train_iou"].append(train_iou)
        history["val_iou"].append(val_iou)

        print(
            f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | "
            f"Train Dice: {train_dice:.4f} | Val Dice: {val_dice:.4f} | "
            f"Train IoU: {train_iou:.4f} | Val IoU: {val_iou:.4f}"
        )

        if val_dice > best_dice:
            best_dice = val_dice
            best_loss = val_loss
            torch.save(model.state_dict(), save_path)
            print(f"✅ Saved best model: {save_path} (Val Dice: {best_dice:.4f})")

    print(f"\nBest Val Dice for {model_name}: {best_dice:.4f} (Loss: {best_loss:.4f})")
    return model, history

# Cell 7: Huấn luyện mô hình đầu tiên (U-Net++) làm baseline chưa augmentation

In [ ]:
import matplotlib.pyplot as plt
import gc

# đảm bảo batch size an toàn cho GPU
CFG.BATCH_SIZE = 2

torch.cuda.empty_cache()
gc.collect()

# Khởi tạo mô hình U-Net++ (nhẹ)
model_unetpp = smp.UnetPlusPlus(
    encoder_name="efficientnet-b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
)

# Huấn luyện
model_unetpp, hist_unetpp = train_model(model_unetpp, model_name="UNetPP_NoAug")

# --- Vẽ biểu đồ ---
epochs = range(1, len(hist_unetpp["train_loss"]) + 1)

plt.figure(figsize=(15,4))

# (1) Loss
plt.subplot(1,3,1)
plt.plot(epochs, hist_unetpp["train_loss"], label="Train Loss")
plt.plot(epochs, hist_unetpp["val_loss"], label="Val Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train vs Val Loss (U-Net++)")
plt.legend()
plt.grid(alpha=0.3)

# (2) Dice
plt.subplot(1,3,2)
plt.plot(epochs, hist_unetpp["train_dice"], label="Train Dice")
plt.plot(epochs, hist_unetpp["val_dice"], label="Val Dice")
plt.xlabel("Epoch")
plt.ylabel("Dice")
plt.title("Train vs Val Dice (U-Net++)")
plt.legend()
plt.grid(alpha=0.3)

# (3) IoU
plt.subplot(1,3,3)
plt.plot(epochs, hist_unetpp["train_iou"], label="Train IoU")
plt.plot(epochs, hist_unetpp["val_iou"], label="Val IoU")
plt.xlabel("Epoch")
plt.ylabel("IoU")
plt.title("Train vs Val IoU (U-Net++)")
plt.legend()
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()
